# SKA-Mid conversion guide

In [1]:
from importlib.metadata import version
import os

try:
    import xradio

    print("XRADIO version", version("xradio"), "already installed.")
except ImportError as e:
    print(e)
    print("Installing XRADIO")

    os.system("pip install xradio")

    import xradio

    print("xradio version", version("xradio"), " installed.")

XRADIO version 0.0.41 already installed.


## Download dataset

In [2]:
import toolviper

toolviper.utils.data.download(file="AA2-Mid-sim_00000.ms")

[2024-09-18 14:30:41,075]  WARNING   toolviper:  File exists: /home/fedemp/ws_xradio_dev/venv_xradio_python312/lib/python3.12/site-packages/toolviper/utils/data/.dropbox 
[2024-09-18 14:30:41,076]     INFO   toolviper:  Updating file metadata information ...  
 

 Download List         
 ────────────────────── 
  AA2-Mid-sim_00000.ms

AA2-Mid-sim_00000.ms.zip:   0%|          | 0.00/14.0M [00:00<?, ?iB/s]

## Convert to Processing Set

In [3]:
from xradio.correlated_data.convert_msv2_to_processing_set import convert_msv2_to_processing_set

ms_file = "AA2-Mid-sim_00000.ms"

main_chunksize = {"frequency": 1, "time": 20}  # baseline, polarization
outfile = "AA2-Mid-sim_00000.vis.zarr"
convert_msv2_to_processing_set(
    in_file=ms_file,
    out_file=outfile,
    parallel=False,
    overwrite=True,
    main_chunksize=main_chunksize,
)

[2024-09-18 14:30:46,158]     INFO   toolviper:  Partition scheme that will be used: ['DATA_DESC_ID', 'OBSERVATION_ID', 'FIELD_ID'] 
[2024-09-18 14:30:46,162]     INFO   toolviper:  Number of partitions: 1 
[2024-09-18 14:30:46,163]     INFO   toolviper:  OBSERVATION_ID [0], DDI [0], STATE [0], FIELD [0], SCAN [0] 
[2024-09-18 14:30:46,466]  WARNING   toolviper:  SOURCE table empty for (unique) source_id [0] and spectral_window_id 0. 


## Processing Set

In [4]:
ps_name = outfile

from xradio.correlated_data.open_processing_set import open_processing_set

ps = open_processing_set(ps_store=outfile)
ps.summary()

,name,intents,shape,polarization,scan_number,spw_name,field_name,source_name,line_name,field_coords,start_frequency,end_frequency
0,AA2-Mid-sim_00000_0,[obs_0],"(95, 1891, 3, 4)","[XX, XY, YX, YY]",[0],spw_0,[_0],[Unknown],[],"[fk5, 0h00m00.00s, -45d00m00.00s]",950000000.0,9.500989e+08


In [6]:
ps["AA2-Mid-sim_00000_0"]

<xarray.Dataset> Size: 35MB
Dimensions:                     (time: 95, baseline_id: 1891, frequency: 3,
                                 polarization: 4, uvw_label: 3)
Coordinates:
    baseline_antenna1_name      (baseline_id) <U5 38kB dask.array<chunksize=(1891,), meta=np.ndarray>
    baseline_antenna2_name      (baseline_id) <U5 38kB dask.array<chunksize=(1891,), meta=np.ndarray>
  * baseline_id                 (baseline_id) int64 15kB 0 1 2 ... 1889 1890
  * frequency                   (frequency) float64 24B 9.5e+08 ... 9.501e+08
  * polarization                (polarization) <U2 32B 'XX' 'XY' 'YX' 'YY'
    scan_number                 (time) int64 760B dask.array<chunksize=(95,), meta=np.ndarray>
  * time                        (time) float64 760B 9.467e+08 ... 9.467e+08
  * uvw_label                   (uvw_label) <U1 12B 'u' 'v' 'w'
Data variables:
    EFFECTIVE_INTEGRATION_TIME  (time, baseline_id) float64 1MB dask.array<chunksize=(20, 1891), meta=np.ndarray>
    FLAG                        (time, baseline_id, frequency, polarization) bool 2MB dask.array<chunksize=(20, 1891, 1, 4), meta=np.ndarray>
    TIME_CENTROID               (time, baseline_id) float64 1MB dask.array<chunksize=(20, 1891), meta=np.ndarray>
    UVW                         (time, baseline_id, uvw_label) float64 4MB dask.array<chunksize=(20, 1891, 3), meta=np.ndarray>
    VISIBILITY                  (time, baseline_id, frequency, polarization) complex64 17MB dask.array<chunksize=(20, 1891, 1, 4), meta=np.ndarray>
    WEIGHT                      (time, baseline_id, frequency, polarization) float32 9MB dask.array<chunksize=(20, 1891, 1, 4), meta=np.ndarray>
Attributes:
    creation_date:     2024-09-18T14:30:46.252883
    data_groups:       {'base': {'correlated_data': 'VISIBILITY', 'flag': 'FL...
    observation_info:  {'observer': ['fred'], 'project': '', 'release_date': ...
    partition_info:    {'field_name': ['_0'], 'intents': ['obs_0'], 'line_nam...
    processor_info:    {'sub_type': '', 'type': ''}
    schema_version:    4.0.-9998
    type:              visibility
    xradio_version:    0.0.41
    antenna_xds:       <xarray.Dataset> Size: 16kB\nDimensions:              ...

In [5]:
ps["AA2-Mid-sim_00000_0"].attrs["antenna_xds"]

<xarray.Dataset> Size: 16kB
Dimensions:                 (antenna_name: 62, cartesian_pos_label: 3,
                             receptor_label: 2, sky_dir_label: 2)
Coordinates:
  * antenna_name            (antenna_name) <U5 1kB 's0000' 's0001' ... 's0061'
  * cartesian_pos_label     (cartesian_pos_label) <U1 12B 'x' 'y' 'z'
    mount                   (antenna_name) <U5 1kB dask.array<chunksize=(62,), meta=np.ndarray>
    polarization_type       (antenna_name, receptor_label) <U1 496B dask.array<chunksize=(62, 2), meta=np.ndarray>
  * receptor_label          (receptor_label) <U5 40B 'pol_0' 'pol_1'
  * sky_dir_label           (sky_dir_label) <U3 24B 'ra' 'dec'
    station                 (antenna_name) <U5 1kB dask.array<chunksize=(62,), meta=np.ndarray>
    telescope_name          (antenna_name) <U35 9kB dask.array<chunksize=(62,), meta=np.ndarray>
Data variables:
    ANTENNA_DISH_DIAMETER   (antenna_name) float64 496B dask.array<chunksize=(62,), meta=np.ndarray>
    ANTENNA_POSITION        (antenna_name, cartesian_pos_label) float64 1kB dask.array<chunksize=(62, 3), meta=np.ndarray>
    ANTENNA_RECEPTOR_ANGLE  (antenna_name, receptor_label) float64 992B dask.array<chunksize=(62, 2), meta=np.ndarray>
Attributes:
    overall_telescope_name:  OSKAR 2.8.4-dev 2023-08-22 d1b86f8c
    relocatable_antennas:    False
    type:                    antenna